# Day 3 - Conversational AI - aka Chatbot!

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import ollama

In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key not set
Google API Key not set


In [7]:
# Initialize

openai = OpenAI()
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [8]:
system_message = "You are a helpful assistant. Just give your answer in briefly."

# Please read this! A change from the video:

In the video, I explain how we now need to write a function called:

`chat(message, history)`

Which expects to receive `history` in a particular format, which we need to map to the OpenAI format before we call OpenAI:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

But Gradio has been upgraded! Now it will pass in `history` in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function `chat(message, history)` where:  
**message** is the prompt to use  
**history** is the past conversation, in OpenAI format  

We will combine the system message, history and latest message, then call OpenAI.

In [12]:
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [13]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant. Just give your answer in briefly.'}, {'role': 'user', 'content': 'hello'}]
History is:
[{'role': 'user', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'hello', 'options': None}, {'role': 'assistant', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'Hello! How can I assist you today?', 'options': None}]
And messages is:
[{'role': 'system', 'content': 'You are a helpful assistant. Just give your answer in briefly.'}, {'role': 'user', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'hello', 'options': None}, {'role': 'assistant', 'metadata': {'title': None, 'id': None, 'parent_id': None, 'duration': None, 'status': None}, 'content': 'Hello! How can I assist you today?', 'options': None}, {'role': 'user', 'content': 'whats Data 

In [14]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # print("History is:")
    # print(history)
    # print("And messages is:")
    # print(messages)

    # Send the request to Ollama
    response = ollama.chat(model=MODEL_LLAMA, messages=messages)
    
    # Extract content from the response
    result = response.message.content
    return result

In [15]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.


In [16]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # print("History is:")
    # print(history)
    # print("And messages is:")
    # print(messages)

    # If Ollama supports streaming, you might handle it like this
    stream = ollama.chat(model=MODEL_LLAMA, messages=messages, stream=True)
    
    response = ""
    for chunk in stream:
        print("Received chunk:")
        # print(chunk)  # Debugging: print the chunk structure
        
        # Access the content directly from the chunk's message
        content = chunk.get('message', {}).get('content', '')
        response += content
        
        # If 'done' is True, end the response stream
        if chunk.get('done', False):
            break
        
        yield response  # Stream the response back incrementally

# gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)


## And then enter Gradio's magic!

In [8]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

In [9]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales evemt.'\
Encourage the customer to buy hats if they are unsure what to get."

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # If Ollama supports streaming, you might handle it like this
    stream = ollama.chat(model=MODEL_LLAMA, messages=messages, stream=True)
    
    response = ""
    for chunk in stream:

        # the content directly from the chunk's message
        content = chunk.get('message', {}).get('content', '')
        response += content
        
        # If 'done' is True, end the response stream
        if chunk.get('done', False):
            break
        
        yield response  # Stream the response back incrementally

In [11]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

In [13]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>

In [17]:
# system message
system_message = (
    "You are a helpful financial assistant for Eclectics Bank Kenya. "
    "Your role is to provide accurate and concise answers, no more than 1 sentence, "
    "to assist customers with inquiries about loans, overdrafts, credit scores, and other banking services. "
    "Always maintain a polite and professional tone, ensuring your replies are easy to understand. "
    "If you don't know the answer, say so and suggest contacting customer service for further assistance and remember to use Kenya shilling Currency.\n\n"
    "Examples:\n"
    "- If asked about loans: 'Eclectics Bank offers a variety of loans with competitive rates, including personal, business, and education loans. Please specify your needs for more details.'\n"
    "- If asked about overdrafts: 'Our overdraft facility helps manage cash flow gaps with dynamic eligibility thresholds based on your account activity.'\n"
    "- If asked about credit scores: 'A good credit score is key to better loan terms. You can improve it by timely payments and reducing debt.'\n\n"
    "Encourage customers to explore Eclectics Bank’s offerings, such as dynamic loan limits, overdrafts, and financial tools tailored to their needs."
)


In [18]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # If Ollama supports streaming, you might handle it like this
    try:
        stream = ollama.chat(model=MODEL_LLAMA, messages=messages, stream=True)
    except Exception as e:
        yield f"An error occurred while starting the chat: {e}"
        return
    
    response = ""
    for chunk in stream:
        try:

            # Extract the content from the chunk's message
            content = chunk.get('message', {}).get('content', '')
            response += content

            # If 'done' is True, end the response stream
            if chunk.get('done', False):
                break

            yield response  # Stream the response back incrementally
        except Exception as e:
            yield f"An error occurred while processing a chunk: {e}"
            break

In [22]:
def chat_gpt(message, history):
    # messages = [
    #     {"role": "system", "content": system_message},
    #     {"role": "user", "content": prompt}
    #   ]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(
        model= MODEL_GPT,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [23]:
gr.ChatInterface(
    fn=chat_gpt,
    type="messages",
    title="Eclectics Banking Assistant Chatbot",
    # description=(
    #     "Welcome to the Banking Assistant Chatbot! This assistant is here to help you with financial queries such as loans, overdrafts, "
    #     "credit scores, and more. Simply type your question, and the assistant will provide concise, professional advice."
    # ),
).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


In [36]:
# Function to provide loan details
def get_loan_details(loan_type):
    loans = {
        'personal': "Personal loans are flexible with competitive rates. Borrow for any personal needs.",
        'business': "Business loans help businesses expand with favorable terms and flexible repayments.",
        'education': "Education loans support tuition and related costs. Flexible repayment options available."
    }
    return loans.get(loan_type, "We offer personal, business, and education loans. Please specify a type.")

# Function to check overdraft eligibility
def overdraft_eligibility(account_balance, transaction_history):
    if account_balance > 10000 and transaction_history > 5:
        return "You are eligible for an overdraft facility based on your account activity."
    else:
        return "Please improve your account balance or transaction history to qualify for an overdraft."

# Function to provide advice on credit score
def credit_score_advice(credit_score):
    if credit_score > 700:
        return "Your credit score is excellent. You qualify for loans with favorable terms."
    elif credit_score > 500:
        return "Your credit score is fair. Consider improving it for better loan terms."
    else:
        return "Your credit score is low. Focus on timely payments and reducing debt to improve it."


In [37]:
loan_function = {
    "name": "get_loan_details",
    "description": "Provides information on different loan types available at Eclectics Bank Kenya.",
    "parameters": {
        "type": "object",
        "properties": {
            "loan_type": {
                "type": "string",
                "description": "The type of loan requested by the customer. Example: personal, business, education."
            },
        },
        "required": ["loan_type"],
        "additionalProperties": False
    }
}

overdraft_function = {
    "name": "overdraft_eligibility",
    "description": "Checks eligibility for overdraft services based on account balance and transaction history.",
    "parameters": {
        "type": "object",
        "properties": {
            "account_balance": {
                "type": "number",
                "description": "The account balance of the customer."
            },
            "transaction_history": {
                "type": "number",
                "description": "The number of transactions the customer has made in the past month."
            },
        },
        "required": ["account_balance", "transaction_history"],
        "additionalProperties": False
    }
}

credit_score_function = {
    "name": "credit_score_advice",
    "description": "Provides credit score advice based on the user's credit score.",
    "parameters": {
        "type": "object",
        "properties": {
            "credit_score": {
                "type": "number",
                "description": "The credit score of the customer."
            },
        },
        "required": ["credit_score"],
        "additionalProperties": False
    }
}

# List of tools (functions) available to the model
tools = [
    {"type": "function", "function": loan_function},
    {"type": "function", "function": overdraft_function},
    {"type": "function", "function": credit_score_function}
]


In [38]:
import json

# Define the system message
system_message = (
    "You are a helpful financial assistant for Eclectics Bank Kenya. "
    "Your role is to provide accurate and concise answers, no more than 1 sentence, "
    "to assist customers with inquiries about loans, overdrafts, credit scores, and other banking services. "
    "Always maintain a polite and professional tone, ensuring your replies are easy to understand. "
    "If you don't know the answer, say so and suggest contacting customer service for further assistance.\n\n"
    "Examples:\n"
    "- If asked about loans: 'Eclectics Bank offers a variety of loans with competitive rates, including personal, business, and education loans. Please specify your needs for more details.'\n"
    "- If asked about overdrafts: 'Our overdraft facility helps manage cash flow gaps with dynamic eligibility thresholds based on your account activity.'\n"
    "- If asked about credit scores: 'A good credit score is key to better loan terms. You can improve it by timely payments and reducing debt.'\n\n"
    "Encourage customers to explore Eclectics Bank’s offerings, such as dynamic loan limits, overdrafts, and financial tools tailored to their needs."
)

In [42]:
def chat(message, history):
    # Combine system message, history, and user input
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # Call the Ollama model to process the input and determine the response
    response = ollama.chat(
        model=MODEL_LLAMA,  
        messages=messages,
        tools=tools 
    )

    print("Response: ===>", response)
    
    # Check if the response contains tool calls
    if 'tool_calls' in response and len(response['tool_calls']) > 0:
        # Extract tool call information
        tool_call = response['tool_calls'][0]
        function_name = tool_call['function']['name']
        arguments = tool_call['function']['arguments']
        
        # Call the function associated with the tool call
        if function_name == "get_loan_details":
            loan_type = arguments.get('loan_type')
            loan_details = get_loan_details(loan_type)  # Define this function to fetch loan details
            
            # Create a response with the loan details and append it to the messages
            tool_response = {
                "role": "tool",
                "content": loan_details
            }
            messages.append(tool_response)

            # Create a final response from the updated messages history
            final_response = ollama.chat(
                model=MODEL_LLAMA,  
                messages=messages
            )

            return final_response['message']['content']
        
        else:
            return "Sorry, I couldn't process the tool call correctly."

    # If no tool call, return the message content
    return response['message']['content']

In [43]:
# Function to handle tool calls
def handle_tool_call(message):
    tool_call = message['tool_calls'][0]
    arguments = json.loads(tool_call['function']['arguments'])
    
    if tool_call['name'] == 'get_loan_details':
        loan_type = arguments.get('loan_type')
        loan_info = get_loan_details(loan_type)
    elif tool_call['name'] == 'overdraft_eligibility':
        account_balance = arguments.get('account_balance')
        transaction_history = arguments.get('transaction_history')
        loan_info = overdraft_eligibility(account_balance, transaction_history)
    elif tool_call['name'] == 'credit_score_advice':
        credit_score = arguments.get('credit_score')
        loan_info = credit_score_advice(credit_score)

    response = {
        "role": "tool",
        "content": json.dumps({"loan_info": loan_info}),
        "tool_call_id": tool_call['id']
    }

    return response, loan_info

In [44]:

gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


Response: ===> model='llama3.2' created_at='2025-01-18T15:49:29.5948177Z' done=True done_reason='stop' total_duration=7004236100 load_duration=54126900 prompt_eval_count=540 prompt_eval_duration=442000000 eval_count=19 eval_duration=6502000000 message=Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='get_loan_details', arguments={'loan_type': 'personal'}))])
